<a href="https://colab.research.google.com/github/Dutra-Apex/AOT-GAN-for-Inpainting/blob/master/mistral-model/Mistral_with_RAG_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook config
Create script so output text wraps

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [2]:
import locale

def getpreferredencoding(do_setlocale = True):
  return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
locale.getdefaultlocale()


('en_US', 'UTF-8')

# Setting up Mistral


## Download and import Libraries

LLM and LangChain libraries

In [3]:
# -q quiets the output
!pip install -qU kaleido python-multipart uvicorn fastapi==0.99.1 typing-extensions==4.5 torch==2.1
!pip install -qU accelerate bitsandbytes langchain
!pip install -qU sentence-transformers chromadb==0.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

Import Libraries

In [4]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import tensorflow
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Download the Mistral 7B Instruct Model and Tokenizer

In [5]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]


## Test Mistral

In [6]:
prompt = """### Instruction: Act as physicist.
Explain what the three body problem is.
 ### Answer:
 """

encoded_instruction = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_instruction.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s> ### Instruction: Act as physicist.
Explain what the three body problem is.
 ### Answer:
  The three-body problem is a complex problem in physics which deals with the motion of three celestial bodies, interacting with each other via gravitational forces. Unlike the two-body problem, where the motion of two objects can be described using the laws of physics, the three-body problem does not have a general, exact solution.

  In the context of celestial mechanics, the three-body problem arises when considering three astronomical objects: a triple star system, a star with a planet in close proximity, or two objects in close proximity to each other, where the gravitational interaction between the three cannot be assumed to be negligible.

  Although it does not have an exact solution, there are several methods available to approximate the solutions of the three-body problem. These include numerical methods such as the Runge-Kutta algorithm, perturbation methods, and Poincaré's limiting s

## Test Mistral without RAG

In [12]:
prompt = """### Instruction: Act as ML engineer.
Explain what the Mamba model is in Machine Learning.
 ### Answer:
 """

encoded_instruction = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_instruction.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> ### Instruction: Act as ML engineer.
Explain what the Mamba model is in Machine Learning.
 ### Answer:
  The Mamba model is not a specific machine learning model with a well-defined meaning in the field of machine learning. Instead, it's a software implementation for the gradient boosting algorithm, particularly for scikit-learn community. Scikit-learn is a popular open-source library for machine learning in Python.

 Mamba is a gradient boosting implementation in scikit-learn that includes Multi-Output, MAnymeaR-Based, parallel, and adaptive Boosting methods. This means it allows handling multiple output variables, can make use of parallel processing, incorporates an adaptive learning process, and builds upon the R `gbm` library for gradient boosting.

 It provides several regression and classification gradient boosting models, including MGBClassifier for classification problems and MGBRegressor for regression problems. You can find more details about Mamba (Gradient Boosting) and

Model halucinates and gives wrong answer to the question. This is expected since the Mamba model being referred is newer tha the Mistral Model, so the LLM has no knowledge of it.

# Integrate LangChain with Mistral

## Create Text Generation Pipeline

In [7]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,  # The higher the temperature, the more 'creative' the model gets
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [8]:
# Create an instance of Mistral
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Create an LLM chain

Source: https://blog.gopenai.com/rag-pipeline-with-mistral-7b-instruct-model-a-step-by-step-guide-138df378a0c2

In [11]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

Source for Mamba article: https://www.unite.ai/mamba-redefining-sequence-modeling-and-outforming-transformers-architecture/

In [16]:
prompt = """
In this article on Mamba, we'll explore how this innovative state-space model (SSM) revolutionizes sequence modeling. Developed by Albert Gu and Tri Dao, Mamba is distinguished for its efficiency in processing complex sequences in fields like language processing, genomics, and audio analysis. Its linear-time sequence modeling with selective state spaces ensures exceptional performance across these diverse modalities.

We'll delve into Mamba's ability to overcome computational challenges faced by traditional Transformers, especially with long sequences. Its selective approach in state space models allows for faster inference and linear scaling with sequence length, significantly improving throughput.

Mamba's uniqueness lies in its rapid processing capability, selective SSM layer, and hardware-friendly design inspired by FlashAttention. These features enable Mamba to outperform many existing models, including those based on the transformer approach, making it a noteworthy advancement in machine learning.

Transformers vs Mamba
Transformers, like GPT-4, have set benchmarks in natural language processing. However, their efficiency dips with longer sequences. Here's where Mamba leaps ahead, with its ability to process long sequences more efficiently and its unique architecture that simplifies the entire process.

Transformers adept at handling sequences of data, such as text for language models. Unlike previous models that processed data sequentially, Transformers process entire sequences simultaneously, enabling them to capture complex relationships within the data.

They use attention mechanism, which allows the model to focus on different parts of the sequence when making predictions.

This attention is computed using three sets of weights: queries, keys, and values, derived from the input data. Each element in a sequence is compared to every other element, providing a weight that signifies the importance, or ‘attention', that each element should receive when predicting the next element in the sequence.

Transformers maintain two main blocks: the encoder, which processes the input data, and the decoder, which generates the output. The encoder consists of multiple layers, each containing two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. Normalization and residual connections are used at each sub-layer to help in training deep networks.

The decoder also has layers with two sub-layers similar to the encoder but adds a third sub-layer that performs multi-head attention over the encoder's output. The sequential nature of the decoder ensures that predictions for a position can only consider earlier positions, preserving the autoregressive property.

In contrast to Transformers, the Mamba model takes a different approach. While Transformers deal with the issue of long sequences by using more complex attention mechanisms, Mamba uses selective state spaces, providing a more comput

Here's a high-level overview of how a transformer functions:

Input Processing: Transformers first encode input data into a format that the model can understand, often using embeddings that also incorporate the position of each element in the sequence.
Attention Mechanism: At its core, the attention mechanism computes a score that represents how much focus to put on other parts of the input sequence when understanding a current element.
Encoder-Decoder Architecture: The transformer model is composed of an encoder to process the input and a decoder to generate the output. Each consists of multiple layers that refine the model's understanding of the input.
Multi-Head Attention: Within both the encoder and decoder, multi-head attention allows the model to simultaneously attend to different parts of the sequence from different representational spaces, improving its ability to learn from diverse contexts.
Position-wise Feed-Forward Networks: After attention, a simple neural network processes the output of each position separately and identically. This is combined with the input through a residual connection and followed by layer normalization.
Output Generation: The decoder then predicts an output sequence, influenced by the encoder's context and what it has generated so far.
The transformer’s ability to handle sequences in parallel and its robust attention mechanism make it powerful for tasks like translation and text generation.

In contrast, the Mamba model operates differently by using selective state spaces to process sequences. This approach addresses the computational inefficiency in Transformers when dealing with lengthy sequences. Mamba's design enables faster inference and scales linearly with sequence length, setting a new paradigm for sequence modeling that could be more efficient, especially as sequences become increasingly lengthy.

Mamba
What makes Mamba truly unique is its departure from traditional attention and MLP blocks. This simplification leads to a lighter, faster model that scales linearly with the sequence length – a feat unmatched by its predecessors.

Key features of Mamba include:

Selective SSMs: These allow Mamba to filter irrelevant information and focus on relevant data, enhancing its handling of sequences. This selectivity is crucial for efficient content-based reasoning.
Hardware-aware Algorithm: Mamba uses a parallel algorithm that's optimized for modern hardware, especially GPUs. This design enables faster computation and reduces the memory requirements compared to traditional models.
Simplified Architecture: By integrating selective SSMs and eliminating attention and MLP blocks, Mamba offers a simpler, more homogeneous structure. This leads to better scalability and performance.
Mamba has demonstrated superior performance in various domains, including language, audio, and genomics, excelling in both pretraining and domain-specific tasks. For instance, in language modeling, Mamba matches or exceeds the performance of larger Transformer models.

Mamba's code and pre-trained models are openly available for community use at GitHub.

Standard Copying tasks are simple for linear models. Selective Copying and Induction Heads require dynamic, content-aware memory for LLMs.
Standard Copying tasks are simple for linear models. Selective Copying and Induction Heads require dynamic, content-aware memory for LLMs.

Structured State Space (S4) models have recently emerged as a promising class of sequence models, encompassing traits from RNNs, CNNs, and classical state space models. S4 models derive inspiration from continuous systems, specifically a type of system that maps one-dimensional functions or sequences through an implicit latent state. In the context of deep learning, they represent a significant innovation, providing a new methodology for designing sequence models that are efficient and highly adaptable.

The Dynamics of S4 Models
SSM (S4) This is the basic structured state space model. It takes a sequence x and produces an output y using learned parameters A, B, C, and a delay parameter Δ. The transformation involves discretizing the parameters (turning continuous functions into discrete ones) and applying the SSM operation, which is time-invariant—meaning it doesn't change over different time steps.

The Significance of Discretization
Discretization is a key process that transforms the continuous parameters into discrete ones through fixed formulas, enabling the S4 models to maintain a connection with continuous-time systems. This endows the models with additional properties, such as resolution invariance, and ensures proper normalization, enhancing model stability and performance. Discretization also draws parallels to the gating mechanisms found in RNNs, which are critical for managing the flow of information through the network.

Linear Time Invariance (LTI)
A core feature of the S4 models is their linear time invariance. This property implies that the model’s dynamics remain consistent over time, with the parameters fixed for all timesteps. LTI is a cornerstone of recurrence and convolutions, offering a simplified yet powerful framework for building sequence models.

Overcoming Fundamental Limitations
The S4 framework has been traditionally limited by its LTI nature, which poses challenges in modeling data that require adaptive dynamics. The recent research paper presents a approach that overcomes these limitations by introducing time-varying parameters, thus removing the constraint of LTI. This allows the S4 models to handle a more diverse set of sequences and tasks, significantly expanding their applicability.

The term ‘state space model' broadly covers any recurrent process involving a latent state and has been used to describe various concepts across multiple disciplines. In the context of deep learning, S4 models, or structured SSMs, refer to a specific class of models that have been optimized for efficient computation while retaining the ability to model complex sequences.

S4 models can be integrated into end-to-end neural network architectures, functioning as standalone sequence transformations. They can be viewed as analogous to convolution layers in CNNs, providing the backbone for sequence modeling in a variety of neural network architectures.

SSM vs SSM + Selection
SSM vs SSM + Selection

Motivation for Selectivity in Sequence Modeling
Structured SSMs
Structured SSMs

The paper argues that a fundamental aspect of sequence modeling is the compression of context into a manageable state. Models that can selectively focus on or filter inputs provide a more effective means of maintaining this compressed state, leading to more efficient and powerful sequence models. This selectivity is vital for models to adaptively control how information flows along the sequence dimension, an essential capability for handling complex tasks in language modeling and beyond.

Selective SSMs enhance conventional SSMs by allowing their parameters to be input-dependent, which introduces a degree of adaptiveness previously unattainable with time-invariant models. This results in time-varying SSMs that can no longer use convolutions for efficient computation but instead rely on a linear recurrence mechanism, a significant deviation from traditional models.

SSM + Selection (S6) This variant includes a selection mechanism, adding input-dependence to the parameters B and C, and a delay parameter Δ. This allows the model to selectively focus on certain parts of the input sequence x. The parameters are discretized taking into account the selection, and the SSM operation is applied in a time-varying manner using a scan operation, which processes elements sequentially, adjusting the focus dynamically over time.

Performance Highlights of Mamba
Mamba is best-in-class on every single evaluation result
Mamba is best-in-class on every single evaluation result

In terms of performance, Mamba excels in both inference speed and accuracy. It's design enables better utilization of longer contexts, which is demonstrated in both DNA and audio modeling, outperforming prior models on complex tasks requiring long-range dependencies. Its versatility is also highlighted in zero-shot evaluations across multiple tasks, setting a new standard for such models in terms of efficiency and scalability.

"""


### Chunk Documents

In [17]:
# Create Document object from text documents
docs = [Document(page_content=post) for post in [prompt]]

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.']
)

document_chunks = text_splitter.split_documents(docs)

In [20]:
print(document_chunks[:2])

[Document(page_content="In this article on Mamba, we'll explore how this innovative state-space model (SSM) revolutionizes sequence modeling. Developed by Albert Gu and Tri Dao, Mamba is distinguished for its efficiency in processing complex sequences in fields like language processing, genomics, and audio analysis. Its linear-time sequence modeling with selective state spaces ensures exceptional performance across these diverse modalities."), Document(page_content="We'll delve into Mamba's ability to overcome computational challenges faced by traditional Transformers, especially with long sequences. Its selective approach in state space models allows for faster inference and linear scaling with sequence length, significantly improving throughput.")]


### Download an Embedding Model

In [14]:
# Downloading embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-zh-v1.5')

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### Initiate a Vector Store Instance

In [21]:
# Initiate a chromadb instance
chroma_db = Chroma.from_documents(document_chunks, embedding_model)
retriever = chroma_db.as_retriever()

### Create Question Answering Chain

In [22]:
# Prompt template
qa_template = """<s>[INST] You are a helpful assistant.
Use the following context to Answer the question below briefly:

{context}

{question} [/INST] </s>
"""

# Create a prompt instance
QA_PROMPT = PromptTemplate.from_template(qa_template)

# Custom QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm = mistral_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT}
)

### Query Mistral 7B Instruct Model

In [23]:
question = "What is the Mamba model?"
response = qa_chain({"query": question})
print(response['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The Mamba model is a state-of-the-art sequence modeling approach developed by Albert Gu and Tri Dao. It differentiates itself from traditional attention and Multi-Layer Perceptron (MLP) block models by adopting a simplified architecture based on selective State-Space Models (SSMs). This design results in improved scalability and performance, allowing Mamba to process complex sequences efficiently in various domains such as language, audio, and genomics. Additionally, Mamba demonstrates superior performance compared to larger Transformer models in language modeling tasks.


In [24]:
question = "Explain what are S4 models and how they work."
response = qa_chain({"query": question})
print(response['result'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


S4 models, also known as Structured State Space models, are a type of sequence model that transform continuous parameters into discrete ones through discretization. They maintain a connection with continuous-time systems, providing additional properties like resolution invariance and proper normalization, which enhance model stability and performance.

At their core, S4 models use the Sequential State Space Model (SSM), which takes a sequence 'x' as input and generates an output 'y'. The model uses learned parameters A, B, C, and a delay parameter Δ. The discretization process converts continuous functions into discrete ones, allowing the application of the time-invariant SSM operation.

Inspired by continuous systems, S4 models map one-dimensional functions or sequences through an implicit latent state. Traditionally, S4 models were limited due to their LTI (Linear Time Invariant) nature, making it challenging to model data requiring adaptive dynamics. However, recent research has int

Model now gives correct answers based on the article